In [12]:
from importlib.metadata import version
print("crewai:", version("crewai"))
print("langchain:", version("langchain"))
print("pydantic:", version("pydantic"))
print("pypdf:", version("pypdf"))

crewai: 0.41.1
langchain: 0.2.13
pydantic: 2.8.2
pypdf: 4.3.1


In [13]:
import os, re, json, time
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, Any, List

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Hardcode OpenAI key (not recommended outside testing!)
os.environ["OPENAI_API_KEY"] = (I inserted my API Key here)

from crewai import Agent, Task, Crew, Process
try:
    from crewai_tools import tool
    TOOL_SOURCE = "crewai_tools"
except ImportError:
    from langchain.tools import tool
    TOOL_SOURCE = "langchain.tools"

print(f"Using tool decorator from: {TOOL_SOURCE}")

DATA_DIR = Path("data"); DATA_DIR.mkdir(exist_ok=True)
OUT_DIR  = Path("outputs"); OUT_DIR.mkdir(exist_ok=True)
MEM_DIR  = Path("memory"); MEM_DIR.mkdir(exist_ok=True)

print("API key set?", bool(os.getenv("OPENAI_API_KEY")))

Using tool decorator from: langchain.tools
API key set? True


In [14]:
#Knowledge Base helpers
def _kb_path(ticker: str) -> Path:
    p = MEM_DIR / f"kb_{ticker.upper()}.jsonl"
    p.touch(exist_ok=True)
    return p

def kb_append_lesson(ticker: str, lesson: dict) -> None:
    p = _kb_path(ticker)
    with p.open("a", encoding="utf-8") as f:
        f.write(json.dumps(lesson, ensure_ascii=False) + "\n")

def kb_load_lessons(ticker: str) -> list[dict]:
    p = _kb_path(ticker)
    out = []
    with p.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                out.append(json.loads(line))
    return out

In [15]:
#Tools
@tool
def load_kb_lessons_tool(ticker: str) -> str:
    """Return past one-sentence takeaways (lessons) for a ticker, one per line."""
    lessons = kb_load_lessons(ticker)
    if not lessons:
        return "(no prior lessons)"
    return "\n".join(f"- {i+1}. {row.get('lesson', '')}" for i, row in enumerate(lessons))

@tool
def save_kb_lesson_tool(ticker: str, lesson: str) -> str:
    """Append a new one-sentence lesson for a ticker to the KB."""
    kb_append_lesson(ticker, {"lesson": lesson, "ts": time.time()})
    return f"Saved lesson for {ticker}: {lesson}"

@tool
def yf_download_tool(ticker: str, period: str = "6mo", interval: str = "1d") -> str:
    """Fetch OHLCV with yfinance and return a compact summary string."""
    df = yf.download(
        ticker, period=period, interval=interval,
        progress=False, group_by="column", auto_adjust=True
    )
    if df.empty:
        return f"{ticker} {period}/{interval}: no data"

    if hasattr(df.columns, "levels") and len(getattr(df.columns, "levels", [])) > 1:
        df.columns = df.columns.get_level_values(0)

    try:
        avg_close = float(df["Close"].mean())
    except Exception:
        avg_close = float(df.filter(regex="(?i)close").mean(numeric_only=True).iloc[0])

    try:
        avg_vol = float(df["Volume"].mean())
    except Exception:
        avg_vol = float(df.filter(regex="(?i)volume").mean(numeric_only=True).iloc[0])

    return (
        f"{ticker} {period}/{interval}: rows={len(df)}, "
        f"avgClose={avg_close:.2f}, avgVol={avg_vol:.0f}"
    )

@tool
def series_snapshot_tool(ticker: str, period: str = "1y", interval: str = "1d") -> str:
    """Return a terse snapshot of recent trend for Close price."""
    df = yf.download(
        ticker, period=period, interval=interval,
        progress=False, group_by="column", auto_adjust=True
    )
    if df.empty:
        return f"{ticker} trend: no data"

    if hasattr(df.columns, "levels") and len(getattr(df.columns, "levels", [])) > 1:
        df.columns = df.columns.get_level_values(0)

    closes = df["Close"].astype(float)
    ret = (closes.iloc[-1] / closes.iloc[0]) - 1.0
    return f"{ticker} trend over {period} @ {interval}: {ret:+.1%}"

In [16]:
#Agent
reviewer = Agent(
    role="Reviewer",
    goal=(
        "Judge the Researcher’s draft strictly, assign a single 1–5 rank, "
        "provide rationale, and produce a concrete improvement plan for the Optimizer. "
        "Store a concise lesson."
    ),
    backstory=(
        "A meticulous investment research QA who grades with a tough rubric "
        "and writes crisp, actionable feedback."
    ),
    tools=[load_kb_lessons_tool, save_kb_lesson_tool, yf_download_tool, series_snapshot_tool],
    allow_delegation=False,
    verbose=True,
)
print("Reviewer agent ready.")

Reviewer agent ready.


In [17]:
#Task
REVIEW_PROMPT = """You are reviewing an investment research draft.

Inputs:
- Ticker: {ticker}
- Draft:
{draft_text}

Context (tools available):
- You may load prior one-sentence takeaways (lessons) for this ticker.
- You may fetch a quick price/volume summary and a simple trend snapshot.

Your job:
1) Assign a grade (integer, 1–5) for overall quality (correctness, completeness, clarity).
2) Write a short rationale (2–5 crisp sentences).
3) Give a concrete improvement plan (3–6 bullet steps).
4) Write exactly one sentence 'lesson' (to save for future reviewers).

Output: STRICT JSON with keys:
{{
  "grade": 1-5 integer,
  "rationale": "string",
  "plan": ["step 1", "step 2", ...],
  "lesson": "one sentence"
}}

Be strict and specific. No text outside the JSON.
"""

review_task = Task(
    description=REVIEW_PROMPT,
    agent=reviewer,
    expected_output='Strict JSON with keys grade, rationale, plan, lesson.',
    output_file=str(OUT_DIR / "reviewer_output.json"),
)
print("Task ready.")

Task ready.


In [18]:
#Crew
crew = Crew(
    agents=[reviewer],
    tasks=[review_task],
    process=Process.sequential,
    verbose=True,
)
print("Crew ready.")

2025-10-03 06:42:04,701 - 27728 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


Crew ready.


In [19]:
#Test run
ticker = "AAPL"
draft_text = """Apple’s services revenue growth is accelerating while hardware cycles normalize.
We expect FY26 EPS upside from margin mix and buybacks. Key risks: China demand and FX."""

print(yf_download_tool.func(ticker, "6mo", "1d"))
print(series_snapshot_tool.func(ticker, "1y", "1d"))

result = crew.kickoff(inputs={"ticker": ticker, "draft_text": draft_text})
print("\n--- RAW RESULT ---\n", result)

AAPL 6mo/1d: rows=127, avgClose=214.76, avgVol=59001226
AAPL trend over 1y @ 1d: +14.8%
 [2025-10-03 06:42:05][DEBUG]: == Working Agent: Reviewer
 [2025-10-03 06:42:05][INFO]: == Starting Task: You are reviewing an investment research draft.

Inputs:
- Ticker: AAPL
- Draft:
Apple’s services revenue growth is accelerating while hardware cycles normalize.
We expect FY26 EPS upside from margin mix and buybacks. Key risks: China demand and FX.

Context (tools available):
- You may load prior one-sentence takeaways (lessons) for this ticker.
- You may fetch a quick price/volume summary and a simple trend snapshot.

Your job:
1) Assign a grade (integer, 1–5) for overall quality (correctness, completeness, clarity).
2) Write a short rationale (2–5 crisp sentences).
3) Give a concrete improvement plan (3–6 bullet steps).
4) Write exactly one sentence 'lesson' (to save for future reviewers).

Output: STRICT JSON with keys:
{
  "grade": 1-5 integer,
  "rationale": "string",
  "plan": ["step 1", 

C:\Users\tommy\anaconda3\envs\crewai-ml\Lib\site-packages\pydantic\main.py:1059: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn(


Thought: I need to load past one-sentence takeaways for Apple (AAPL) to assess the draft based on historical context.

Action: load_kb_lessons_tool
Action Input: {"ticker": "AAPL"} 

- 1. Ensure investment research drafts are supported with recent data and comprehensive analysis of market conditions.

Thought: I have obtained a prior lesson indicating the importance of using recent data and comprehensive market analysis. Now, I should fetch a recent price/volume summary and trend snapshot for AAPL to evaluate the draft's accuracy and completeness.

Action: yf_download_tool
Action Input: {"ticker": "AAPL", "period": "6mo", "interval": "1d"} 

AAPL 6mo/1d: rows=127, avgClose=214.76, avgVol=59001387

Thought: I have gathered the necessary information, including past lessons and current data, to evaluate the draft for AAPL.

Final Answer: 
{
  "grade": 3,
  "rationale": "The draft touches on key aspects such as services revenue growth and potential EPS upside from margin mix and buybacks, 

In [20]:
#Save lesson from the result JSON and show KB
def _extract_json(obj) -> dict:
    if isinstance(obj, dict):
        return obj
    try:
        return json.loads(str(obj))
    except Exception:
        s = str(obj)
        start = s.find("{")
        end = s.rfind("}")
        if start != -1 and end != -1 and end > start:
            return json.loads(s[start:end+1])
        raise

payload = _extract_json(result)
lesson = payload.get("lesson", "").strip()
if lesson:
    save_kb_lesson_tool.func(ticker, lesson)
    print("KB now:")
    for row in kb_load_lessons(ticker):
        print("-", row["lesson"])
else:
    print("No 'lesson' field found in the result.")

KB now:
- Ensure investment research drafts are supported with recent data and comprehensive analysis of market conditions.
- Investment drafts should pair qualitative insights with quantitative data to strengthen analysis.


In [21]:
print(load_kb_lessons_tool.func(ticker))

- 1. Ensure investment research drafts are supported with recent data and comprehensive analysis of market conditions.
- 2. Investment drafts should pair qualitative insights with quantitative data to strengthen analysis.
